In [2]:
from google.colab import files
uploaded = files.upload()

Saving Hitters (1).csv to Hitters (1).csv


In [3]:
import os, glob
glob.glob("*")


['Hitters (1).csv', 'sample_data']

In [4]:
import numpy as np
import pandas as pd

# Reproducibility
np.random.seed(42)

# Parameters
n_samples = 500
n_features = 7
base_corr = 0.9

# Correlated features
cov = np.full((n_features, n_features), base_corr)
np.fill_diagonal(cov, 1.0)
mean = np.zeros(n_features)
X = np.random.multivariate_normal(mean, cov, size=n_samples)

# Linear target
true_weights = np.array([2.5, -1.8, 1.2, 0.8, 0.5, 1.5, -0.7])
bias = 3.0
noise = np.random.normal(0, 1.5, size=n_samples)
y = X @ true_weights + bias + noise

# DataFrame
cols = [f"Feature_{i+1}" for i in range(n_features)]
df = pd.DataFrame(X, columns=cols)
df["Target"] = y

# Quick check + save
print(df[cols].corr().round(3))
df.to_csv("highly_correlated_dataset.csv", index=False)
print("Saved: highly_correlated_dataset.csv")


           Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
Feature_1      1.000      0.898      0.899      0.895      0.894      0.895   
Feature_2      0.898      1.000      0.904      0.915      0.903      0.916   
Feature_3      0.899      0.904      1.000      0.907      0.891      0.893   
Feature_4      0.895      0.915      0.907      1.000      0.902      0.895   
Feature_5      0.894      0.903      0.891      0.902      1.000      0.890   
Feature_6      0.895      0.916      0.893      0.895      0.890      1.000   
Feature_7      0.903      0.908      0.906      0.903      0.904      0.895   

           Feature_7  
Feature_1      0.903  
Feature_2      0.908  
Feature_3      0.906  
Feature_4      0.903  
Feature_5      0.904  
Feature_6      0.895  
Feature_7      1.000  
Saved: highly_correlated_dataset.csv


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Use the correlated features we generated above (df) OR synthesize here if you skipped Cell C
if 'df' in globals():
    X = df[[c for c in df.columns if c.startswith("Feature_")]].values
    y = df["Target"].values
else:
    # quick synthetic fallback
    np.random.seed(42)
    n = 1200
    x1 = np.random.randn(n)
    X = np.column_stack([x1 + 0.01*np.random.randn(n) for _ in range(7)])
    y = 3*x1 + 0.5*np.random.randn(n)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

def ridge_cost(Xb, y, w, lam):
    yhat = Xb @ w
    mse = np.mean((yhat - y)**2)
    l2 = lam * np.sum(w[1:]**2)
    return mse + l2

def ridge_gd(X, y, lr=1e-3, lam=1.0, epochs=2000):
    Xb = add_bias(X)
    w = np.zeros(Xb.shape[1])
    n = Xb.shape[0]
    for _ in range(epochs):
        yhat = Xb @ w
        grad = (2/n) * (Xb.T @ (yhat - y))
        grad[1:] += 2*lam*w[1:]
        # Check for NaN in gradients and weights, and break if found
        if np.isnan(grad).any() or np.isnan(w).any():
            break
        w -= lr * grad
    return w

# Adjusted learning rates and lambda values
lrs  = [1e-5, 1e-4, 1e-3, 1e-2]
lams = [10**-15, 10**-10, 10**-5, 10**-3, 0, 1]

best = {"cost": np.inf, "r2": -np.inf, "lr": None, "lam": None, "w": None}
Xb_tr, Xb_te = add_bias(X_train), add_bias(X_test)

for lr in lrs:
    for lam in lams:
        w = ridge_gd(X_train, y_train, lr=lr, lam=lam, epochs=1500)
        # Check if weights are valid before calculating cost and R2
        if w is not None and not np.isnan(w).any():
            c = ridge_cost(Xb_tr, y_train, w, lam)
            r2 = r2_score(y_test, Xb_te @ w)
            if (c < best["cost"]) or (np.isclose(c, best["cost"]) and r2 > best["r2"]):
                best.update({"cost": c, "r2": r2, "lr": lr, "lam": lam, "w": w})

print("Best -> lr:", best["lr"], "lambda:", best["lam"])
print("Train cost:", round(best["cost"],6))
print("Test  R2 :", round(best["r2"],6))
print("Test  MSE:", mean_squared_error(y_test, Xb_te @ best["w"]))

Best -> lr: 0.01 lambda: 0
Train cost: 2.344395
Test  R2 : 0.893491
Test  MSE: 2.2710594339350716


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

# Ensure the filename matches exactly what you uploaded
csv_name = "Hitters (1).csv"
df = pd.read_csv(csv_name)

# Clean
target_col = 'Salary' if 'Salary' in df.columns else df.columns[-1]
df = df.dropna(subset=[target_col]).copy()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

X = df.drop(columns=[target_col])
y = df[target_col].astype(float)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

pre = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

alpha = 0.5748
pipe_lin   = Pipeline([("pre", pre), ("mdl", LinearRegression())])
pipe_ridge = Pipeline([("pre", pre), ("mdl", Ridge(alpha=alpha, random_state=42))])
pipe_lasso = Pipeline([("pre", pre), ("mdl", Lasso(alpha=alpha, random_state=42, max_iter=10000))])

pipe_lin.fit(X_train, y_train)
pipe_ridge.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)

def eval_model(name, pipe):
    y_pred = pipe.predict(X_test)
    print(f"{name:>8} | R2={r2_score(y_test, y_pred):.4f} | MSE={mean_squared_error(y_test, y_pred):.2f}")

eval_model("Linear", pipe_lin)
eval_model("Ridge",  pipe_ridge)
eval_model("LASSO",  pipe_lasso)

scores = {
    "Linear": r2_score(y_test, pipe_lin.predict(X_test)),
    "Ridge":  r2_score(y_test, pipe_ridge.predict(X_test)),
    "LASSO":  r2_score(y_test, pipe_lasso.predict(X_test))
}
print("Best by R2:", max(scores, key=scores.get))


  Linear | R2=0.2907 | MSE=128284.35
   Ridge | R2=0.3003 | MSE=126562.19
   LASSO | R2=0.3009 | MSE=126447.81
Best by R2: LASSO


/tmp/ipython-input-925499817.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [8]:
import numpy as np
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Try legacy load_boston else OpenML fallback
try:
    from sklearn.datasets import load_boston
    boston = load_boston()
    X_b, y_b = boston.data, boston.target
except Exception:
    from sklearn.datasets import fetch_openml
    boston = fetch_openml(name="boston", version=1, as_frame=True)
    X_b, y_b = boston.data.values, boston.target.astype(float).values

X_tr, X_te, y_tr, y_te = train_test_split(X_b, y_b, test_size=0.2, random_state=0)

alphas = np.logspace(-4, 2, 30)

ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(X_tr, y_tr)
y_pred_r = ridge_cv.predict(X_te)
print("RidgeCV alpha:", ridge_cv.alpha_)
print("RidgeCV  R2:", round(r2_score(y_te, y_pred_r), 4), "MSE:", round(mean_squared_error(y_te, y_pred_r), 2))

lasso_cv = LassoCV(alphas=alphas, max_iter=10000, random_state=0)
lasso_cv.fit(X_tr, y_tr)
y_pred_l = lasso_cv.predict(X_te)
print("LassoCV alpha:", lasso_cv.alpha_)
print("LassoCV  R2:", round(r2_score(y_te, y_pred_l), 4), "MSE:", round(mean_squared_error(y_te, y_pred_l), 2))


RidgeCV alpha: 0.04893900918477494
RidgeCV  R2: 0.5886 MSE: 33.5
LassoCV alpha: 0.0001
LassoCV  R2: 0.5892 MSE: 33.45


In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target

sc = StandardScaler()
X = sc.fit_transform(X)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def bin_logreg_fit(X, y, lr=0.1, lam=1e-3, epochs=3000):
    Xb = np.c_[np.ones((X.shape[0], 1)), X]
    w = np.zeros(Xb.shape[1])
    n = Xb.shape[0]
    for _ in range(epochs):
        z = Xb @ w
        p = sigmoid(z)
        grad = (1/n) * (Xb.T @ (p - y))
        grad[1:] += 2*lam*w[1:]
        w -= lr * grad
    return w

def bin_logreg_predict_proba(X, w):
    Xb = np.c_[np.ones((X.shape[0], 1)), X]
    return sigmoid(Xb @ w)

classes = np.unique(y_tr)
weights = []
for c in classes:
    yc = (y_tr == c).astype(float)
    w = bin_logreg_fit(X_tr, yc, lr=0.1, lam=1e-3, epochs=3000)
    weights.append(w)
weights = np.stack(weights)

probas = np.column_stack([bin_logreg_predict_proba(X_te, w) for w in weights])
y_pred = np.argmax(probas, axis=1)
print("OvR Logistic Accuracy:", round(accuracy_score(y_te, y_pred), 4))


OvR Logistic Accuracy: 0.9333
